In [1]:
# Reuse DSA code from previous

q = 0xf4f47f05794b256174bba6e9b396a7707e563c5b
p_hex = '800000000000000089e1855218a0e7dac38136ffafa72eda7' \
        '859f2171e25e65eac698c1702578b07dc2a1076da241c76c6' \
        '2d374d8389ea5aeffd3226a0530cc565f3bf6b50929139ebe' \
        'ac04f48c3c84afb796d61e5a4f9a8fda812ab59494232c7d2' \
        'b4deb50aa18ee9e132bfa85ac4374d7f9091abc3d015efc87' \
        '1a584471bb1'
g_hex = '5958c9d3898b224b12672c0b98e06c60df923cb8bc999d119' \
        '458fef538b8fa4046c8db53039db620c094c9fa077ef389b5' \
        '322a559946a71903f990f1f7e0e025e2d7f7cf494aff1a047' \
        '0f5b64c36b625a097f1651fe775323556fe00b3608c887892' \
        '878480e99041be601a62166ca6894bdd41a7054ec89f756ba' \
        '9fc95302291'

p = int(p_hex, base = 16)
g = int(g_hex, base = 16)

N = 40*4
L = len(p_hex)*4
assert (p-1) % q == 0

In [2]:
from hashlib import sha256

# Create a hash function with |H| = N bits output
def Hash(x: bytes, N = N):
    assert type(x) is bytes
    assert N <= 256
    digest_bytes = sha256(x).digest()
    digest_int = int.from_bytes(digest_bytes, byteorder = 'big')
    return digest_int & ( ( 0b1 << N ) - 1 )

assert Hash(b'abc') < 2**N

In [3]:
from random import randint

# Commenting out some assert statments because I think that's the point of the attack

def DSAGeneratePrivateKey(p = p, q = q, g = g):
    return randint(1, q-1)

def DSAGeneratePublicKey(x, p = p, q = q, g = g ):
    assert type(x) is int and 1 <= x <= q-1
    return pow(g, x, p)

def DSASign(message, x, p = p, q = q, g = g, H = Hash):
    # x is the privkey
    assert type(message) is bytes
    assert type(x) is int and 1 <= x <= q-1
    
    k = randint(1, q-1)
    r = pow(g, k, p) % q
    
    k_inv = pow(k, q-2, q) # https://en.wikipedia.org/wiki/Fermat%27s_little_theorem
    assert (k*k_inv) % q == 1
    s = ( k_inv*( H(message) + x*r  ) ) % q
    # assert s != 0 and r != 0
    
    
    signature = dict(r = r, s = s)
    return signature
    

def DSAVerify(message, r, s, y, p = p, q = q, g = g, H = Hash):
    # y is the pubkey
    assert type(message) is bytes
    # assert 0 < r < q 
    # assert 0 < s < q
    
    w  = pow(s, q-2, q) # w is s inverse, again using FLT
    assert w*s % q == 1
    u1 = w*H(message) % q
    u2 = w*r % q
    
    # Using property (A * B) mod C = (A mod C * B mod C) mod C
    v  = pow(g, u1, p) * pow(y, u2, p)
    v %= p
    v %= q
    
    return True if v == r else False

def ModInv(x, p):
    # Using Fermat's little theorem
    y = pow(x, p-2, p)
    assert ( x*y ) % p == 1
    return y

In [4]:
# Now, execute the attack with g = 0

privkey = DSAGeneratePrivateKey(g = 0)
pubkey  = DSAGeneratePublicKey(privkey, g = 0)

message = b'Hello, world!'
signature = DSASign(message, privkey, g = 0)

# r = 0 in this signature, which breaks a lot of the important mathematic properties
print('Signature: r = {r}, s = {s}'.format(**signature))

# The signature verifies...
assert DSAVerify(message, y = pubkey, **signature, g = 0)

# But it also verifies any other string!!!
assert DSAVerify(message = b'Goodbye, world.', y = pubkey, **signature, g = 0)


Signature: r = 0, s = 7506716411092445803742951501475697472135831025


In [5]:
# Now, execute the attack with g = p+1

privkey = DSAGeneratePrivateKey(g = p+1)
pubkey  = DSAGeneratePublicKey(privkey, g = p+1)

# Generate an arbitrary message
message = b'sdkfjalskdjflaksdjfl'

# Now forge a signature... delete the privkey to prove it's not being used
del(privkey)

z = Hash(message)
r = pow(pubkey, z, p) % q
s = ( r * ModInv(z, p) ) % q

# Show that it validates
assert DSAVerify(message, y = pubkey, r = r, s = s, g = p+1)